<a href="https://colab.research.google.com/github/cyun-404/capston/blob/main/generation_deploy_colab1_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [1]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 2.8 MB 12.1 MB/s 
     |████████████████████████████████| 57 kB 7.1 MB/s 
     |████████████████████████████████| 582 kB 53.1 MB/s 
     |████████████████████████████████| 419 kB 52.5 MB/s 
     |████████████████████████████████| 120 kB 68.2 MB/s 
     |████████████████████████████████| 3.3 MB 46.0 MB/s 
     |████████████████████████████████| 880 kB 65.2 MB/s 
     |████████████████████████████████| 96 kB 4.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=dc6f472d0952c2ee4c47a31ef24abf0324cdfaf75d4498d145d7a8ba702e7329
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstallin

# 모델 로딩
프리트레인한 GPT2 모델과 토크나이저를 읽어 들입니다.

In [2]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval()

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [3]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


## 프롬프트 준비

언어모델에 넣을 프롬프트를 준비합니다.

In [4]:
input_ids = tokenizer.encode("기준금리", return_tensors="pt")

## Greedy Search


다음 단어 확률 분포에서 최대 확률을 내는 단어들을 리턴합니다. 
여러 번 수행하더라도 생성 결과가 바뀌지 않습니다 (`do_sample=False`).
`max_length`는 생성 최대 길이이며 이보다 길거나, 짧더라도 EOD 토큰 등 스페셜 토큰이 나타나면 생성을 중단합니다. `min_length`는 생성 최소 길이이며 이보다 짧은 구간에서 EOD 등 스페셜 토큰이 등장해 생성이 중단될 경우 해당 토큰이 나올 확률을 0으로 수정하여 문장 생성이 종료되지 않도록 강제합니다.

In [5]:
import torch
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 대한 기대감이 커지면서 채권금리가 상승했다.
이날 3년 만기 국고채 금리는 전날보다 0.07%포인트 오른 연 5.86%로 마감했다.
5년 만기 국고채 금리는 연 5.86%로 0.07


## Beam Search 

Beam Search는 다음 단어 확률 분포에서 `num_beams`만큼의 경우의 수를 남겨가면서 문장을 생성합니다. Beam search는 Greedy search보다 계산량이 많지만 좀 더 확률값이 높은 문장을 생성할 수 있습니다.

In [6]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        num_beams=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 대한 기대감이 커지며 채권금리가 급등했다.
이날 채권시장에서 3년 만기 국고채 금리는 전날보다 0.07%포인트 오른 연 5.38%로 마감했다.
5년 만기 국고채 금리도 0.07%


`num_beams=1`로 설정하면 정확히 Greedy search와 동일하게 작동합니다.


In [7]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        num_beams=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 대한 기대감이 커지면서 채권금리가 상승했다.
이날 3년 만기 국고채 금리는 전날보다 0.07%포인트 오른 연 5.86%로 마감했다.
5년 만기 국고채 금리는 연 5.86%로 0.07


## 반복 줄이기

### 반복되는 n-gram 사이즈를 지정하기

위의 예시를 보면 `"그럼, 그건 뭐예요?"`이 반복됩니다. 이를 아래와 같이 지정해 반복을 방지합니다. 3개 이상의 토큰이 반복될 경우 해당 3-gram 등장 확률을 0으로 만들어 생성 결과에서 배제합니다.


In [8]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        no_repeat_ngram_size=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 대한 기대감이 커지면서 채권금리가 상승했다.
이날 3년 만기 국고채 금리는 전날보다 0.07%포인트 오른 연 5.86%로 마감했다.
5년 만기 국고채는 0.06%포인트 상승한 연 5.9


### repetition penalty

repetition penalty로 반복을 통제할 수도 있습니다. 다음과 같이 실행하면 되며 그 범위는 1 이상의 값을 가져야 합니다. 1이라면 아무런 패널티를 적용하지 않는게 됩니다.


In [9]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 대한 기대감이 커지면서 채권금리가 상승했다.
이날 3년 만기 국고채 금리는 전날보다 0.07%포인트 오른 연 5.86%로 마감했다.
5년 만기 국고채 금리는 연 5.86%로 0.07


`repetition_penalty` 값이 클 수록 패널티가 세게 적용됩니다.

In [10]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 대한 기대감이 커지면서 채권금리가 상승했다.
이날 3년 만기 국고채 금리는 전날보다 0.07%포인트 오른 연 5.86%로 마감했다.
5년 만기 국고채 금리도 0.04%포인트 오른 연 6.9


In [11]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.2,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 대한 기대감이 커지면서 채권금리가 상승했다.
이날 3년 만기 국고채 금리는 전날보다 0.07%포인트 오른 연 5.86%로 마감됐다.
5년물과 10년물도 각각 0.15%, 0.29% 올랐다.
국고


In [12]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 대한 기대감이 커지면서 채권금리가 상승했다.
이날 3년 만기 국고채 금리는 전날보다 0.07%포인트 오른 연 5.86%로 마감됐다.
5년물과 10년물도 각각 0.15%, 0.29%를 기록했다.
국고


## top-k sampling

지금까지는 생성을 반복하더라도 그 결과가 동일한 샘플링 방식을 살펴봤습니다. top-k sampling은 다음 단어를 뽑을 때 확률값 기준 가장 큰 k개 가운데 하나를 선택하는 기법입니다. 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, k개 안에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있습니다. 따라서 top-k sampling은 매 시행 때마다 생성 결과가 달라집니다. k는 1 이상의 값을 지녀야 합니다.

In [13]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 따른 부작용을 줄이기 위해 저조한 예금금리를 끌어올리고 있다.
이에 따라 연초 정기예금 금리는 대부분 연 3%대였으니 저금리 상태가 계속될 경우 연 2%까지 떨어진다는 분석이 나온다.
1인당 정기


k=1일 경우 Greedy search와 동일합니다. 

In [14]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 대한 기대감이 커지면서 채권금리가 상승했다.
이날 3년 만기 국고채 금리는 전날보다 0.07%포인트 오른 연 5.86%로 마감했다.
5년 만기 국고채 금리는 연 5.86%로 0.07


## top-k sampling + temperature scaling

top-k sampling은 temperature scaling과 동시에 적용할 수 있습니다. 그 값에 따라 다음과 같은 효과가 납니다.

(1) t가 0에 가까워질 수록 토큰 분포가 sharp해진다 > 1등 토큰이 뽑힐 확률이 그만큼 높아진다 > do_sample=True이지만 사실상 greedy decoding이 된다

In [15]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 따른 자금 유출입 변동성이 확대될 수 있다는 우려도 나온다.
한은은 이날 금융통화위원회를 열고 기준금리를 연 1.25%로 0.25%포인트 인상했다.
지난해 11월 0.25%포인트 인상한 이후 5개월 만에


(2) t=1이라면 모델 출력 분포를 그대로 사용한다 > 하지만 샘플링 방식을 사용하기 때문에 생성할 때마다 다른 문장이 나온다

In [16]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상 등 영향으로 최근 금값이 큰 폭 하락한 반면, 달러화나 유로 등 주요 통화에 비해 절하폭은 작아 상승 여력이 크다고 이 연구원은 전망했다.
삼성증권은 이날 보고서에서 "금리 상승에 따라 수익과 위험을 고려한 자산


(3) t를 키울수록 토큰 분포가 uniform해진다 > 사실상 uniform sampling이 된다, 생성 품질이 악화할 가능성이 높아진다

In [17]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=100000000.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 2.127%, 예대증률 3.515. 3년물은 마나트 오일인 점을 언급하면서, 그러나 현재 시중 물가는 4조~68%에 비해 과도한 가격수준을 나타내고 있음이라고도 부시의 말 한마디로 시장 안정을 어렵게 하였다.
특히


## top-p sampling

top-p sampling은 다음 단어를 뽑을 때 누적 확률값이 p 이하인 단어들 가운데 하나를 선택하는 기법입니다. 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, 누적 확률값 p 이하에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있습니다. 따라서 top-p sampling은 매 시행 때마다 생성 결과가 달라집니다. p는 확률이기 때문에 0~1 사이의 값을 지녀야 합니다. p가 1이라면 어휘 집합에 있는 모든 단어를 대상으로 샘플링하기 때문에 top-p sampling 효과가 사라집니다. p가 1보다 약간 작다면 확률값이 낮은 일부 단어들을 다음 단어 후보에서 제거해 생성 품질을 높입니다. 

In [18]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 따른 수요 증가를 고려할 때 부동산 시장의 회복은 필연적"이라고 말했다.
하지만 일각에서는 부동산 경기침체가 주택 시장의 전반적인 침체를 불러왔다는 지적이 나오고 있다.
주택건설업체인 우미건설은 올들어 서울 지역 분양물량이


p가 0에 가까울 경우 Greedy search와 비슷해 집니다.

In [19]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상에 대한 기대감이 커지면서 채권금리가 상승했다.
이날 3년 만기 국고채 금리는 전날보다 0.07%포인트 오른 연 5.86%로 마감했다.
5년 만기 국고채 금리는 연 5.86%로 0.07


## 통합 적용

저희가 실습에 사용하고 있는 허깅페이스(huggingface) 라이브러리의 구현상 적용 순서는 다음과 같습니다.

- _get_logits_processor
  - RepetitionPenalty
  - NoRepeatNGramLogits
  - MinLengthLogits
- _get_logits_warper
  - TemperatureLogits
  - TopKLogits
  - TopPLogits

유효한 설정들을 종합 적용해 문장을 생성하는 코드는 다음과 같습니다. 샘플링(top-k, top-p)을 적용하기 때문에 시행 때마다 다른 문장이 생성됩니다.

In [20]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=200,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        temperature=0.9,
        top_k=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

기준금리 인상으로 가계부채 부담이 늘고 있다.
전문가들은 금리 인상 시점이 올해 3분기가 돼야 가능할 것으로 보고 있다.</d> 국내 금융권 채용시장에서 구직자가 선호하는 스펙으로는 △금융사 관련 자격증 취득(21.3%) ▲IT·ICT 기술 전문가(23.6%)가 선정됐다.
신입사원이라면 자기소개서 전형시 ‘스펙’보다는 ‘신상 및 자격증(20.8%), 기타’(7.4) 항목을 더 중요하게 고려하는 상황이다.
특히 신입 지원자들의 경우 서류전형과 면접을 모두 통과할 수 있는 가장 확실한 기준이 직무능력이라고 응답했다.
잡코리아가 취업준비생들을 대상으로 한 설문조사를 통해 진행한 설문조사 결과에 따르면 “직무능력과 자신의 능력을 기준으로 인재를 뽑는다는 기업들의 취지와 달리 최근에는 다양한 분야에 대한 수요가 늘면서 이에 맞춰 각 분야의 역량검증도 중요해지고 있다”며 “이 같은 인재 선호 경향 속에 많은 기업들이 경력에 필요한 역량을 쌓기 위해 노력하고 있지만, 현재로선 이러한 노력의 결과가 나오기 어렵다”고 밝혔다.
또
